In [8]:
from google.cloud import storage
from google.cloud import bigquery
import os
import pandas as pd
import decimal

import tqdm
from io import StringIO
import numpy as np

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/myles/ficc_myles/eng-reactor-287421-1f272e8b3a32.json"
bq_client = bigquery.Client()
table_id = 'eng-reactor-287421.mergent.AGENT'

schema_dict = {'agent': [bigquery.SchemaField("action","string"), bigquery.SchemaField("agent_id_l","numeric"), bigquery.SchemaField("legal_name_c","string")],
               'coupchst': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("change_date_d","date"), bigquery.SchemaField("new_coupon_rate_f","numeric")],
               'varrate': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("initial_interest_rate_f","numeric"), bigquery.SchemaField("initial_interest_rate_date_d","date"), bigquery.SchemaField("current_interest_rate_f","numeric"), bigquery.SchemaField("reset_frequency_i","string"), bigquery.SchemaField("next_variable_rate_date_d","date"), bigquery.SchemaField("reset_day_c","string"), bigquery.SchemaField("reset_month_c","string"), bigquery.SchemaField("floor_f","numeric"), bigquery.SchemaField("ceiling_f","numeric"), bigquery.SchemaField("rate_calc_method_code_i","string"), bigquery.SchemaField("interest_payment_day_code_c","string"), bigquery.SchemaField("note_c","string"), bigquery.SchemaField("instrument_type_c","string"), bigquery.SchemaField("rate_type_c","string"), bigquery.SchemaField("determination_date_d","date"), bigquery.SchemaField("effective_date_d","date"), bigquery.SchemaField("valid_until_date_d","date"), bigquery.SchemaField("formula_c","string"), bigquery.SchemaField("fix_float_ind_i","string"), bigquery.SchemaField("benchmark_c","string")],
               'varhist': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("next_variable_rate_date_d","date"), bigquery.SchemaField("current_interest_rate_f","numeric"), bigquery.SchemaField("determination_date_d","date"), bigquery.SchemaField("effective_date_d","date"), bigquery.SchemaField("valid_until_date_d","date"), bigquery.SchemaField("formula_c","string"), bigquery.SchemaField("fix_float_ind_i","string")],
               'issuagnt': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("agent_id_l","numeric"), bigquery.SchemaField("agent_role_c","string")],
               'addlcred': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("addl_credit_schedule_num_l","numeric"), bigquery.SchemaField("addl_credit_type_code_c","string"), bigquery.SchemaField("row_number_l","numeric"),	bigquery.SchemaField("effective_date_d","date"), bigquery.SchemaField("expiration_date_d","date"), bigquery.SchemaField("has_guarantees_i","string"), bigquery.SchemaField("agent_id_l","numeric")],
               'bondinfo': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("cusip_c","string"), bigquery.SchemaField("coupon_f","numeric"), bigquery.SchemaField("maturity_date_d","date"), bigquery.SchemaField("settlement_date_d","date"),	bigquery.SchemaField("maturity_amount_f","numeric"), bigquery.SchemaField("series_code_c","string"), bigquery.SchemaField("active_maturity_flag_i","string"), bigquery.SchemaField("coupon_code_c","string"), bigquery.SchemaField("debt_type_c","string"), bigquery.SchemaField("offering_price_f","numeric"), bigquery.SchemaField("offering_yield_f","numeric"), bigquery.SchemaField("total_maturity_offering_amt_f","numeric"), bigquery.SchemaField("tot_mat_amt_outstanding_f","numeric"), bigquery.SchemaField("tot_mat_amt_outstanding_date_d","date"), bigquery.SchemaField("additional_credit_flag_i","string"), bigquery.SchemaField("addl_credit_schedule_num_i","numeric"), bigquery.SchemaField("series_c","string"), bigquery.SchemaField("default_flag_i","string"), bigquery.SchemaField("dfrd_int_cnvrsn_date_d","date"), bigquery.SchemaField("put_flag_i","string"),	bigquery.SchemaField("optional_call_flag_i","string"), bigquery.SchemaField("call_schedule_number_l","numeric"), bigquery.SchemaField("redemption_flag_i","string"), bigquery.SchemaField("prtl_redemption_flag_i","string"), bigquery.SchemaField("reoffered_i","string"), bigquery.SchemaField("reoffered_yield_f","numeric"), bigquery.SchemaField("reoffered_date_d","date"), bigquery.SchemaField("material_event_flag_i","string"), bigquery.SchemaField("capital_purpose_c","string"), bigquery.SchemaField("tax_code_c","string"), bigquery.SchemaField("state_tax_i","string"), bigquery.SchemaField("bank_qualified_i","string"), bigquery.SchemaField("orig_cusip_status_i","string"), bigquery.SchemaField("orig_cusip_type_i","string"), bigquery.SchemaField("prior_cusip_c","string"), bigquery.SchemaField("cusip_change_reason_c","string"), bigquery.SchemaField("cusip_change_date_d","date"), bigquery.SchemaField("project_name_c","string"), bigquery.SchemaField("use_of_proceeds_c","string"),	bigquery.SchemaField("security_code_i","string"), bigquery.SchemaField("sink_fund_type_i","string"), bigquery.SchemaField("super_sinker_flag_i","string"), bigquery.SchemaField("registration_type_i","string"), bigquery.SchemaField("average_life_date_d","date"), bigquery.SchemaField("dated_date_d","date"), bigquery.SchemaField("delivery_date_d","date"),	bigquery.SchemaField("interest_calc_code_i","string"), bigquery.SchemaField("first_coupon_date_d","date"), bigquery.SchemaField("interest_frequency_i","string"), bigquery.SchemaField("interest_accrual_date_d","date"), bigquery.SchemaField("depository_type_i","string"), bigquery.SchemaField("denomination_amount_f","numeric"), bigquery.SchemaField("bond_insurance_code_c","string"),	bigquery.SchemaField("mtg_insurance_code_c","string"), bigquery.SchemaField("moody_long_rating_c","string"), bigquery.SchemaField("moody_long_date_d","string"), bigquery.SchemaField("moody_short_rating_c","string"), bigquery.SchemaField("moody_short_date_d","string"), bigquery.SchemaField("moody_conditional_c","string"), bigquery.SchemaField("sp_long_rating_c","string"), bigquery.SchemaField("sp_long_date_d","string"), bigquery.SchemaField("sp_short_rating_c","string"), bigquery.SchemaField("sp_short_date_d","string"), bigquery.SchemaField("sp_provisional_c","string"), bigquery.SchemaField("fitch_long_rating_c","string"), bigquery.SchemaField("fitch_long_date_d","string"), bigquery.SchemaField("fitch_short_rating_c","string"), bigquery.SchemaField("fitch_short_date_d","string"), bigquery.SchemaField("fitch_conditional_c","string"), bigquery.SchemaField("update_date_d","date"), bigquery.SchemaField("note_c","string"), bigquery.SchemaField("isin_c","string"),	bigquery.SchemaField("cav_maturity_amount_f","numeric"), bigquery.SchemaField("next_par_call_date_d","date"), bigquery.SchemaField("next_par_call_price_f","numeric"), bigquery.SchemaField("next_call_date_d","date"), bigquery.SchemaField("next_call_price_f","numeric"), bigquery.SchemaField("sf_accel_pct_f","numeric"), bigquery.SchemaField("make_whole_call_flag_i","string"), bigquery.SchemaField("make_whole_start_date_d","date"), bigquery.SchemaField("make_whole_end_date_d","date"), bigquery.SchemaField("make_whole_spread_l","numeric"), bigquery.SchemaField("make_whole_benchmark_c","string"), bigquery.SchemaField("make_whole_text_c","string"), bigquery.SchemaField("redeem_method_c","string"), bigquery.SchemaField("sinking_fund_allocation_i","string"), bigquery.SchemaField("source_of_repayment_i","string"), bigquery.SchemaField("next_int_pay_date_d","string"), bigquery.SchemaField("last_int_pay_date_d","string"), bigquery.SchemaField("rule_144a_i","string"), bigquery.SchemaField("seniority_c","string")],
               'issuinfo': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("issuer_long_name_c","string"), bigquery.SchemaField("state_c","string"), bigquery.SchemaField("issue_description_c","string"), bigquery.SchemaField("settlement_type_c","string"), bigquery.SchemaField("settlement_status_i","string"), bigquery.SchemaField("gross_spread_f","numeric"), bigquery.SchemaField("selling_concession_f","numeric"),	bigquery.SchemaField("reallowance_f","numeric"), bigquery.SchemaField("offering_type_c","string"), bigquery.SchemaField("total_offering_amount_f","numeric"), bigquery.SchemaField("offering_date_d","date"), bigquery.SchemaField("first_coupon_date_d","date"), bigquery.SchemaField("interest_payment_day_c","string"), bigquery.SchemaField("interest_payment_month_c","string"), bigquery.SchemaField("active_issue_i","string"), bigquery.SchemaField("reoffered_i","string"), bigquery.SchemaField("special_opt_call_code_c","string"), bigquery.SchemaField("special_mand_call_code_c","string"), bigquery.SchemaField("extraordinary_call_flag_i","string"), bigquery.SchemaField("call_at_cav_indr_i","string"), bigquery.SchemaField("whole_call_frequency_i","string"), bigquery.SchemaField("part_call_frequency_i","string"), bigquery.SchemaField("optional_call_note_c","string"), bigquery.SchemaField("corporate_backer_l","numeric"), bigquery.SchemaField("call_notice_days_l","numeric"), bigquery.SchemaField("issuer_short_name_c","string"), bigquery.SchemaField("issuer_note_c","string"), bigquery.SchemaField("incremental_denomination_f","numeric"), bigquery.SchemaField("total_issue_amount_out_f","numeric"), bigquery.SchemaField("green_bond_i","string")],
               'redemptn': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("redemption_type_i","string"), bigquery.SchemaField("redemption_date_d","date"), bigquery.SchemaField("redemption_price_f","numeric"), bigquery.SchemaField("redemption_amt_f","numeric"), bigquery.SchemaField("redemption_rate_at_cav_f","numeric"), bigquery.SchemaField("refunding_cusip_c","string"), bigquery.SchemaField("escrow_type_i","string"), bigquery.SchemaField("calls_defeased_flag_i","string"), bigquery.SchemaField("sink_defeased_flag_i","string"), bigquery.SchemaField("refunding_issue_dtd_d","date"), bigquery.SchemaField("ref_issue_settlement_date_d","date"), bigquery.SchemaField("note_c","string"), bigquery.SchemaField("escrow_percentage_c","string")],
               'partredm': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("partial_call_type_i","string"), bigquery.SchemaField("partial_call_date_d","date"), bigquery.SchemaField("row_number_l","numeric"), bigquery.SchemaField("partial_call_rate_f","numeric"), bigquery.SchemaField("prtl_call_rate_at_cav_f","numeric"), bigquery.SchemaField("prtl_call_amt_f","numeric"), bigquery.SchemaField("prtl_call_amt_at_cav_f","numeric"), bigquery.SchemaField("source_agent_l","numeric"), bigquery.SchemaField("note_c","string")],
               'sinkfund': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("sink_date_d","date"), bigquery.SchemaField("sink_price_f","numeric"), bigquery.SchemaField("skip_payment_i","string"), bigquery.SchemaField("sink_amt_at_maturity_f","numeric"),	bigquery.SchemaField("sink_amt_at_cav_f","numeric"), bigquery.SchemaField("note_c","string")],
               'issudflt': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("default_date_d","date"), bigquery.SchemaField("default_type_i","string"), bigquery.SchemaField("default_event_type_c","string"), bigquery.SchemaField("default_event_date_d","date"), bigquery.SchemaField("default_status_c","string"), bigquery.SchemaField("default_status_date_d","date"), bigquery.SchemaField("default_source_c","string"), bigquery.SchemaField("default_source_date_d","date"), bigquery.SchemaField("default_note_c","string"), bigquery.SchemaField("reinstated_i","string"), bigquery.SchemaField("reinstated_date_d","date")],
               'ratingsh': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("cusip_c","string"), bigquery.SchemaField("rating_type_c","string"), bigquery.SchemaField("rating_c","string"), bigquery.SchemaField("creditwatch_c","string"), bigquery.SchemaField("rating_date_d","date"), bigquery.SchemaField("outlook_c","string")],           
               'ratncode': [bigquery.SchemaField("action","string"),bigquery.SchemaField("type_c","string "), bigquery.SchemaField("code_c","string "), bigquery.SchemaField("description_c","string ")],
               'code': [bigquery.SchemaField("action","string "),bigquery.SchemaField("code_id_c","string "), bigquery.SchemaField("code_col_name_c","string "), bigquery.SchemaField("code_c","string"), bigquery.SchemaField("code_desc_c","string ")],
               'agentadr': [bigquery.SchemaField("action","string"),bigquery.SchemaField("agent_id_l","numeric"), bigquery.SchemaField("address_id_l","numeric"), bigquery.SchemaField("addr1_c","string"), bigquery.SchemaField("addr2_c","string"), bigquery.SchemaField("city_c","string"), bigquery.SchemaField("state_c","string"), bigquery.SchemaField("zipcode_c","string")],
               'ratings': [bigquery.SchemaField("action","string"),bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("cusip_c","string"), bigquery.SchemaField("rating_type_c","string"), bigquery.SchemaField("rating_c","string"), bigquery.SchemaField("creditwatch_c","string"), bigquery.SchemaField("rating_date_d","date"), bigquery.SchemaField("outlook_c","string")],
               'bondaddl': [bigquery.SchemaField("action","string"),bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("next_int_pay_date_d","date"), bigquery.SchemaField("last_int_pay_date_d","date")],
               'varschd': [bigquery.SchemaField("action","string"),bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("start_date_d","date"), bigquery.SchemaField("end_date_d","date"), bigquery.SchemaField("formula_c","string")],
               'callschd':[bigquery.SchemaField("action","STRING"), bigquery.SchemaField("issue_id_l","NUMERIC"), bigquery.SchemaField("maturity_id_l", "NUMERIC"), bigquery.SchemaField("call_schedule_number_l", "NUMERIC"), bigquery.SchemaField("call_date_d","DATE"), bigquery.SchemaField("call_price_f","NUMERIC")],
               'putsched': [bigquery.SchemaField("action","string"), bigquery.SchemaField("issue_id_l","numeric"), bigquery.SchemaField("maturity_id_l","numeric"), bigquery.SchemaField("put_or_tender_type_i","string"), bigquery.SchemaField("put_or_tender_date_d","date"), bigquery.SchemaField("put_or_tender_price_f","numeric"), bigquery.SchemaField("put_or_tender_frequency_i","string"), bigquery.SchemaField("next_put_or_tender_date_d","date"), bigquery.SchemaField("final_put_or_tender_date_d","date"), bigquery.SchemaField("put_or_tender_window1_l","numeric"), bigquery.SchemaField("put_or_tender_window2_l","numeric"), bigquery.SchemaField("put_fee_f","numeric"), bigquery.SchemaField("note_c","string")]
              }

In [9]:


def hello_gcs(event, context):
    import re
    from google.cloud import storage
    from google.cloud import bigquery
    from google.cloud.exceptions import NotFound

   

In [10]:
def get_table_name_from_file_name(file_name):    
    start = file_name.find('/',5)
    end = file_name.find('.UPD')
    return (file_name[start+1:end].lower())

In [11]:
temp = None

In [12]:

def load_mergent_file_to_bq(file_name,bucket):    
    blob = bucket.blob(file_name)
    data_str = blob.download_as_string()
    data = str(data_str,'utf-8')
    data = StringIO(data)
    table_name = get_table_name_from_file_name(file_name)
    col_names = [schema_dict[table_name][i].name for i in range(len(schema_dict[table_name]))]
    col_names.append('Junk')
    df=pd.read_csv(data, sep='|',names=col_names,index_col=False, usecols=col_names[:-1])
    df.to_csv('temp.csv',index=False)
    for col in schema_dict[table_name]:
        if col.field_type.upper() == 'NUMERIC':
            df[col.name] = df[col.name].astype(str).map(decimal.Decimal)
        elif col.field_type.upper() == 'DATE':
            df[col.name] = pd.to_datetime(df[col.name].astype(str),format='%Y%m%d').dt.date
        if col.field_type.upper() == 'STRING':
            df[col.name] = df[col.name].astype(str)
            
    job_config = bigquery.LoadJobConfig(
    schema=schema_dict[table_name] ,
    write_disposition="WRITE_APPEND")

    job = bq_client.load_table_from_dataframe(df, 'eng-reactor-287421.mergent.' + table_name.upper(), job_config=job_config) 

    job.result()
    

    

In [13]:
load_mergent_file_to_bq('2020/meod1218/PARTREDM.UPD',storage.Client().bucket("mergent_data"))

In [14]:
bucket = storage.Client().bucket("mergent_data")
folders_list = ['2019','2020','2021','munifull']
for folder in folders_list:
    for blob in tqdm.tqdm(bucket.list_blobs(prefix=folder),total=len([upd for upd in bucket.list_blobs(prefix=folder)])):
        file_name = blob.name
        if file_name.find('.DS') == -1:
            try:
                load_mergent_file_to_bq(blob.name,bucket)
            except Exception as e:
                print(file_name)
        else:
            continue


  2%|▏         | 23/1143 [02:05<1:50:20,  5.91s/it]/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (8,56,85,86) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
 40%|████      | 460/1143 [35:55<45:09,  3.97s/it]  /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
 45%|████▌     | 515/1143 [39:54<29:39,  2.83s/it]  

2019/meod1111/BONDINFO.UPD


 58%|█████▊    | 666/1143 [50:39<32:53,  4.14s/it]  

2019/meod1121/SINKFUND.UPD


 60%|█████▉    | 685/1143 [52:05<22:52,  3.00s/it]

...

munifull/SINKFUND.DLM


 93%|█████████▎| 26/28 [03:26<00:09,  4.65s/it]

munifull/VARHIST.DLM


100%|██████████| 28/28 [03:26<00:00,  7.39s/it]

munifull/VARRATE.DLM
munifull/VARSCHED.DLM


In [15]:
print([upd for upd in bucket.list_blobs()])

[<Blob: mergent_data, /daily_files/meod1013.zip, 1634242724250603>, <Blob: mergent_data, /daily_files/munieod_2019.zip, 1634242721716931>, <Blob: mergent_data, /daily_files/munieod_2020.zip, 1634242715371821>, <Blob: mergent_data, /daily_files/munieod_2021.zip, 1634242742209012>, <Blob: mergent_data, /daily_files/munifull.zip, 1634242691167894>, <Blob: mergent_data, 2019/.DS_Store, 1634232036915882>, <Blob: mergent_data, 2019/meod1001/ADDLCRED.UPD, 1634232036364476>, <Blob: mergent_data, 2019/meod1001/AGENT.UPD, 1634232036043268>, <Blob: mergent_data, 2019/meod1001/AGENTADR.UPD, 1634232034940702>, <Blob: mergent_data, 2019/meod1001/BONDADDL.UPD, 1634232035693976>, <Blob: mergent_data, 2019/meod1001/BONDINFO.UPD, 1634232034937714>, <Blob: mergent_data, 2019/meod1001/CALLSCHD.UPD, 1634232035659202>, <Blob: mergent_data, 2019/meod1001/COUPCHST.UPD, 1634232036683727>, <Blob: mergent_data, 2019/meod1001/ISSUAGNT.UPD, 1634232035928909>, <Blob: mergent_data, 2019/meod1001/ISSUDFLT.UPD, 163423